# titanic with RF -- uis cs570 mike

always try to FIRST get a generalized prediction ASAP.

just dont waste time on model/features/EDA.

this wil be a benchmark.

improve models and use error metric.

- this is a fast model
- ignoring categorical variables initially

In [97]:
from sklearn.ensemble import RandomForestRegressor

# for ROC-AUC error metric.
from sklearn.metrics import roc_auc_score
import pandas as pd


X = pd.read_csv("train.csv")
y = X.pop("Survived")

In [98]:
# describe will show only numeric and not categorical variables.
X.describe()
#X.info()
#X.head()
#X.shape

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [99]:
# age has missing values. so impute it
X["Age"].fillna(X.Age.mean(), inplace=True)

X.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [100]:
# Selecting only numeric variables
filter_dtype = (X.dtypes != "object")
numeric_vars = list(X.dtypes[filter_dtype].index)
X[numeric_vars].head(10)

# as of now PID is worthless but it might happen that it is related to the order in which passengers booked tickets

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,1,3,22.000000,1,0,7.2500
1,2,1,38.000000,1,0,71.2833
2,3,3,26.000000,0,0,7.9250
3,4,1,35.000000,1,0,53.1000
4,5,3,35.000000,0,0,8.0500
5,6,3,29.699118,0,0,8.4583
6,7,1,54.000000,0,0,51.8625
7,8,3,2.000000,3,1,21.0750
8,9,3,27.000000,0,2,11.1333
9,10,2,14.000000,1,0,30.0708


In [101]:
# building the model now
# n_estimators chosen 100 because dataset is small
# out of bag score helps capture validation
model = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=42)

# using only num_vars
model.fit(X[numeric_vars], y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [102]:
# for regression oob_score_ attribute gives R^2 based on oob predictions
# ATTRIBUTES HAVING TRAILING UNDERSCORE ARE ONLY AVAILABLE AFTER TRAINING
model.oob_score_

0.1361695005913669

In [103]:
# to calculate ROC-AUC
y_oob = model.oob_prediction_

print("C-STAT or roc/auc = ", roc_auc_score(y, y_oob))
print("==============This is our FIRST BENCHMARK==============")

C-STAT or roc/auc =  0.7399551550399983
==============This is our FIRST BENCHMARK==============


In [104]:
# creating a function to get descriptive statistics for categorical variables

def describe_categorical(X):
    from IPython.display import display, HTML
    cat_columns = X.columns[X.dtypes == "object"]
    display(HTML(X[cat_columns].describe().to_html()))

In [105]:
describe_categorical(X)

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Taylor, Mrs. Elmer Zebley (Juliet Cummins Wright)",male,CA. 2343,C23 C25 C27,S
freq,1,577,7,4,644


In [121]:
# dropping few variables for my ease
X.drop(["Name", "Ticket", "PassengerId"], axis=1, inplace=True)
print(X.head(10))

   Pclass     Sex        Age  SibSp  Parch     Fare Cabin Embarked
0       3    male  22.000000      1      0   7.2500   NaN        S
1       1  female  38.000000      1      0  71.2833   C85        C
2       3  female  26.000000      0      0   7.9250   NaN        S
3       1  female  35.000000      1      0  53.1000  C123        S
4       3    male  35.000000      0      0   8.0500   NaN        S
5       3    male  29.699118      0      0   8.4583   NaN        Q
6       1    male  54.000000      0      0  51.8625   E46        S
7       3    male   2.000000      3      1  21.0750   NaN        S
8       3  female  27.000000      0      2  11.1333   NaN        S
9       2  female  14.000000      1      0  30.0708   NaN        C


In [129]:
# changing the cabin variable to be only the first letter....CREATE A FUNCTION ON COLUMN using ---> APPLY()

    
def clean_cabin(x):
    try:
        return x[0]
    except TypeError:
        return "None"

X["Cabin"] = X.Cabin.apply(clean_cabin)
X.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.000000,1,0,7.2500,N,S
1,1,female,38.000000,1,0,71.2833,C,C
2,3,female,26.000000,0,0,7.9250,N,S
3,1,female,35.000000,1,0,53.1000,C,S
4,3,male,35.000000,0,0,8.0500,N,S
5,3,male,29.699118,0,0,8.4583,N,Q
6,1,male,54.000000,0,0,51.8625,E,S
7,3,male,2.000000,3,1,21.0750,N,S
8,3,female,27.000000,0,2,11.1333,N,S
9,2,female,14.000000,1,0,30.0708,N,C
